
---

### This 📘 notebook needs to be run from VS Code with [these prerequisites](https://github.com/johnmaeda/SK-Recipes#-prerequisites-to-getting-started-with-the-notebooks-in-this-repo).

How to use this notebook: 

* Just read the text and scroll along until you run into code blocks.
* Code blocks have computer code inside them — hover over the block and you can run the code.
* Run the code by hitting the ▶️ "play" button to the left. If the code runs you'll see a ✔️. If not, you'll get a ❌.
* The output and status of the code block will appear just below itself — you need to scroll down further to see it.
* Sometimes a code block will ask you for input in a hard-to-notice dialog box 👆 at the top of your notebook window. 

---

# S1E1 • 🔥 Kernel Starter Notebook

This notebook has been designed to get you started quickly with Semantic Kernel — the easy way to add LLM AI to your app. It's in three parts:

1. **Setting up your OpenAI or Azure OpenAI Service key.** This lets you use this notebook like a playground of sorts. And you only have to enter your key once to get going — it stores it locally into a file called "settings.json" on your disk. ⚠️ Be sure to not let that file show up publicly anywhere like on your personal GitHub repo — so please .gitignore it.

2. **Getting a kernel instantiated.** With your OpenAI or Azure OpenAI key you can then create a kernel to send instructions to. We've made it easy for you to use either OpenAI or Azure OpenAI. When using OpenAI, it will default to your using the `text-davinci-003` model; when you use Azure OpenAI there's an extra endpoint setting to consider — and in addition you're asked explicitly for the model you would like to use.

3. **Run a semantic function.** Okay! You're ready to give your LLM AI a natural language prompt expressed as natural language. We call this kind of interaction with the model "semantic" because it lives in the world of the underlying meaning of the text you give to the model. 

## Step 1️: Set up your OpenAI or Azure OpenAI Service key

### Step 1.1: The system will ask you to type in your secret API key.

The boolean variable below `useAzureOpenAI` should be set to `false` if you are using a regular OpenAI key, and `true` if you have an Azure OpenAI Service key. Whichever way you set the variable, a rectangular box will appear at the 👆 top 👆 of the notebook page — asking you to enter the credentials. These credentials are stored locally on your own disk and don't go anywhere else. Once you hit the ▶️ button to run the code, be sure to 👆 look up to the top of this notebook page to enter your creds.

You will only see the ▶️ (play) when you hover over the code block.

In [ ]:
#!import ../config/Settings.cs

bool useAzureOpenAI = false;

await Settings.AskAzureEndpoint(useAzureOpenAI);
await Settings.AskModel(useAzureOpenAI);
await Settings.AskApiKey(useAzureOpenAI);

If you can see a confirmation line above that goes `Settings: OK: AI model configured ...` then proceed to step 2. Note that this is done only once. And if you do it incorrectly, then use the instructions below in 1.2 to 🔴 reset your locally stored credentials. If your credentials are good to go, DO NOT reset your locally stored credentials.

You just need to do this **once**. And the rest of the notebooks will use the API key that is stored on your computer in this one big, important step.

### Step 1.2: 🔴 To reset your locally stored credentials ...

_SKIP THIS STEP if you successfully set your key in step 1.1._

Only press the ▶️ button below if you want to reset what you entered above. In order to do so, you will need to "uncomment" two characters by removing `//` at the start of line 4.

In other words, change `// Settings.Reset();` to `Settings.Reset();`. By doing so you are "uncommenting" that line of code so it becomes active. That line is "commented" — meaning it's deactivated — so that normally the code block will not work. We set it to be deactivated by default so that you don't inadvertently set your API key in step 1.1 and then just erase it in this step (1.2) by accident.

In [ ]:
#!import ../config/Settings.cs

// Uncomment the line below to reset your settings and run step 1.1 again so it asks you for your API key
// Settings.Reset();

## Step 2️: Get a 🔥 kernel ready for you to cook up your recipe

Congratulations! You're one-third of the way there! Hit ▶️ below to access the locally stored credentials you set up in the first step. This step loads the Microsoft.SemanticKernel package and gets the rest of the notebook prepared to get your semantic function running asap.

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.8.56.1-preview"

#!import ../config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.KernelExtensions;
using System.IO;
using Microsoft.SemanticKernel.Configuration;
using Microsoft.SemanticKernel.SemanticFunctions;

IKernel kernel = Microsoft.SemanticKernel.Kernel.Builder.Build();

// Grab the locally stored credentials from the settings.json file. Name the "backend" as "davinci" — assuming that you're using one of the davinci completion models. 

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    kernel.Config.AddAzureOpenAICompletionBackend("davinci", model, azureEndpoint, apiKey);
else
    kernel.Config.AddOpenAICompletionBackend("davinci", model, apiKey, orgId);

If you can see a confirmation line above that starts with `Installed Packages ... • Microsoft.SemanticKernel, ...` then proceed to step 3.

😱 **Get an error message?** The most common error is accidentally resetting your credentials file, or setting it up with the wrong information. Look at the 'settings.json' file in your config directory to make sure your settings make sense. And if you're still stuck, go to https://aka.ms/sk/discord where we have realtime support.

## Step 3️: Run a semantic 🧂 function in Semantic Kernel

Before you set off to write a semantic function, review our documentation on semantic functions at our [learning hub](https://learn.microsoft.com/en-us/semantic-kernel/howto/semanticfunctions). You'll want to feel comfortable with two ideas:

* Creating a parameterized prompt with one variable — `$input` is the default input variable — that you can learn more about [here](https://learn.microsoft.com/semantic-kernel/howto/semanticfunctions#writing-a-more-powerful-templated-prompt).

* Configuring your prompt with a few standard settings — `MaxTokens`, `Temperature`, `TopP` — that you can learn more about [here](https://learn.microsoft.com/en-us/semantic-kernel/howto/configuringfunctions).

You are just three more steps to running a semantic function. Get ready!

### Step 3.1: Define a parameterized prompt that takes a single input

In [ ]:
string skPrompt = """
{{$input}}

Summarize the content above in less than 140 characters.
""";

### Step 3.2: Tune the prompt to have a more deterministic or less deterministic behavior

The `MaxTokens` setting determines how much processing latitude you're giving to the model — the smaller it is, the less likely it will be to complete your ask. It's the single most important [setting](https://learn.microsoft.com/en-us/semantic-kernel/howto/configuringfunctions) for you to know because it impacts how much you are spending with each request.

Also, you can subtly shape the output of the response with the other two parameters. To make the response more or less "creative," tweak the `Temperature` setting between 0 (straight ball) and 1 (curve ball). You can also set the `TopP` setting between 0 (smaller vocabulary) and 1 (larger vocabulary) for a different kind of result.

In [ ]:
using Microsoft.SemanticKernel.KernelExtensions;

var promptConfig = new PromptTemplateConfig
{
    Completion =
    {
        MaxTokens = 1000, Temperature = 0.2, TopP = 0.5,
    }
};

var promptTemplate = new PromptTemplate(
    skPrompt, promptConfig, kernel
);

var functionConfig = new SemanticFunctionConfig(promptConfig, promptTemplate);

var summaryFunction = kernel.RegisterSemanticFunction("MySkill", "Summary", functionConfig);


### Step 3.3: Set your input to the templated prompt and have the kernel 🔥 process it

In [ ]:

var input = """
I think with some confidence I can say that 2023 is going to be the most exciting year that 
the AI community has ever had,” writes Kevin Scott, chief technology officer at Microsoft, 
in a Q&A on the company’s AI blog. He acknowledges that he also thought 2022 was the most 
exciting year for AI, but he believes that the pace of innovation is only increasing. 
This is particularly true with generative AI, which doesn’t simply analyze large data sets 
but is a tool people can use to create entirely new works. We can already see its promise 
in systems like GPT-3, which can do anything from helping copyedit and summarize text to 
providing inspiration, and DALL-E 2, which can create useful and arresting works of art 
based on text inputs. Here are some of Scott’s predictions about how AI will change the 
way we work and play.
""";
// Text source: https://www.microsoft.com/en-us/worklab/kevin-scott-on-5-ways-generative-ai-will-transform-work-in-2023

var summary = await kernel.RunAsync(input, summaryFunction);

Console.WriteLine(summary);

🎉 You've made your first Semantic Kernel semantic function. Congratulations 🔥!

🤔 **Get `"Error: Throttling: Too many requests ..."` message?** The OpenAI services turn out to be extremely popular these days. If you're using the key for a free account, this message will pop up often. 

😱 **Get a different error message?** If you can't see a summarization of the text above, then go to https://aka.ms/sk/discord where we have realtime support available to troubleshoot your problem.

# ⏭️ Next Steps

Run through more advanced examples in the notebooks that are available in our GitHub repo at [https://aka.ms/sk/repo](https://aka.ms/sk/repo).

Or stay a longer while and change the prompt above to your liking; and also the `$input` and other parameters to your liking. Please keep in mind that each API call to OpenAI or Azure OpenAI Services will use up tokens.